<a href="https://colab.research.google.com/github/priyank96/idl-spring-22-project-deepar/blob/main/DL_project_basic_model_all_data_no_vol_with_context_shreya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/priyank96/idl-spring-22-project-deepar

fatal: destination path 'idl-spring-22-project-deepar' already exists and is not an empty directory.


In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import logging
from sklearn import preprocessing
import torch.optim as optim
from tqdm import tqdm
import os
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
!python3 /content/idl-spring-22-project-deepar/src/data/company_data.py

In [ ]:
import pickle
DATA_PATH = '/content/idl-spring-22-project-deepar/data'

with open(DATA_PATH+'/index_to_company.pkl','rb') as f:
  index_to_company = pickle.load(f)

with open(DATA_PATH+'/company_to_index.pkl','rb') as f:
  company_to_index = pickle.load(f)

In [ ]:
print(index_to_company)
print(company_to_index)

{0: 'FCX', 1: 'FTCH', 2: 'TGT', 3: 'CRWD', 4: 'CRON', 5: 'MGM', 6: 'CRSM', 7: 'PRU', 8: 'CLOV', 9: 'BTCUSD', 10: 'NEM', 11: 'FXE', 12: 'VRM', 13: 'TER', 14: 'WCLD', 15: 'AVCT', 16: 'GOLD', 17: 'NICH', 18: 'MMM', 19: 'VUZI', 20: 'LMND', 21: 'AMGN', 22: 'CCIV', 23: 'CBAT', 24: 'APT', 25: 'LUV', 26: 'QYLD', 27: 'SYK', 28: 'LPTX', 29: 'AA', 30: 'CFG', 31: 'JDST', 32: 'BLUE', 33: 'AEO', 34: 'YHOO', 35: 'SOXX', 36: 'OIH', 37: 'NIO', 38: 'NXPI', 39: 'GGG', 40: 'UPS', 41: 'SBGI', 42: 'WFC', 43: 'FAMI', 44: 'CVNA', 45: 'DIS', 46: 'MAG', 47: 'E', 48: 'ETRM', 49: 'CRTX', 50: 'GOOS', 51: 'XERS', 52: 'HPE', 53: 'BYND', 54: 'ARKK', 55: 'DLTR', 56: 'CS', 57: 'PAR', 58: 'SHAK', 59: 'KWEB', 60: 'GM', 61: 'RCRT', 62: 'CL', 63: 'SOS', 64: 'LOCO', 65: 'VZ', 66: 'IMCC', 67: 'XNET', 68: 'ZGNX', 69: 'CNY', 70: 'G', 71: 'MCMJ', 72: 'EVO', 73: 'DB', 74: 'LC', 75: 'CLPT', 76: 'GWSO', 77: 'DOCU', 78: 'TWLO', 79: 'NET', 80: 'A', 81: 'HLT', 82: 'ALDX', 83: 'SOFI', 84: 'INPX', 85: 'PINS', 86: 'DIA', 87: 'HOG', 88: 

In [ ]:
DATA_PATH = '/content/idl-spring-22-project-deepar/data'

stock_inputs = np.load(DATA_PATH + '/stock_inputs.npy', allow_pickle=True)
stock_labels = np.load(DATA_PATH + '/stock_labels.npy', allow_pickle=True)

stock_test_inputs = np.load(DATA_PATH + '/stock_test_inputs.npy',allow_pickle=True)
stock_test_labels = np.load(DATA_PATH + '/stock_test_labels.npy',allow_pickle=True)

In [ ]:
NUM_TRAIN_SAMPLES = 500000
NUM_TEST_SAMPLES  = 100000

stock_inputs_trimmed = stock_inputs[:NUM_TRAIN_SAMPLES,:,:]
stock_labels_trimmed = stock_labels[:NUM_TRAIN_SAMPLES,:]

stock_test_inputs_trimmed = stock_test_inputs[:NUM_TEST_SAMPLES,:,:]
stock_test_labels_trimmed = stock_test_labels[:NUM_TEST_SAMPLES,:]

In [ ]:
print('stock_inputs_trimmed shape', stock_inputs_trimmed.shape)
print('stock_labels_trimmed shape', stock_labels_trimmed.shape)
print('stock_test_inputs_trimmed shape', stock_test_inputs_trimmed.shape)
print('stock_test_labels_trimmed shape', stock_test_labels_trimmed.shape)

stock_inputs_trimmed shape (500000, 192, 5)
stock_labels_trimmed shape (500000, 192)
stock_test_inputs_trimmed shape (100000, 192, 5)
stock_test_labels_trimmed shape (100000, 192)


In [ ]:
# On the Filtered input, check the number of actual companies retained

train_comp_ids = set()
test_comp_ids  = set()

# Every Seq in the window will belong to the same company
# Hence seq_id = 0
# cov_id = -1 (last index)
seq_id = 0
cov_id = -1

for sample in range(0, NUM_TRAIN_SAMPLES):
  train_comp_ids.add(stock_inputs_trimmed[sample][seq_id][cov_id])

for sample in range(0, NUM_TEST_SAMPLES):
  test_comp_ids.add(stock_test_inputs_trimmed[sample][seq_id][cov_id])

In [ ]:
print(sorted(train_comp_ids))
print(len(train_comp_ids))
print(sorted(test_comp_ids))
print(len(test_comp_ids))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158

In [ ]:
params = {
    'num_classes': len(train_comp_ids),
    'embedding_dim':128,
    'cov_dim': 3,
    'lstm_hidden_dim': 256,
    'lstm_layers':4 ,
    'window_size':192,
    'batch_size': 32,
    'learning_rate': 2e-3,
    'epochs':100,
    'num_test_samples': NUM_TEST_SAMPLES,
    'num_train_samples': NUM_TRAIN_SAMPLES,
    'conditioning_period': 168,
    'prediction_period': 24,
    'context_length':10
}

print(params)

{'num_classes': 249, 'embedding_dim': 128, 'cov_dim': 3, 'lstm_hidden_dim': 256, 'lstm_layers': 4, 'window_size': 192, 'batch_size': 32, 'learning_rate': 0.002, 'epochs': 100, 'num_test_samples': 100000, 'num_train_samples': 500000, 'conditioning_period': 168, 'prediction_period': 24, 'context_length': 10}


In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, Sampler
from pathlib import Path
import sys

DATA_PATH = '/content/idl-spring-22-project-deepar/data'


class TrainDataset(Dataset):
    def __init__(self):
        self.data = stock_inputs_trimmed
        #Delete Volume
        self.data = np.delete(self.data, 1, axis=2)
        self.label = stock_labels_trimmed
        self.train_len = self.data.shape[0]


    def __len__(self):
        return self.train_len
  
    def __getitem__(self, index):
        x1 = torch.from_numpy(self.data[index].astype(np.float32))
        x2 = torch.from_numpy(self.label[index].astype(np.float32))
        return x1, x2  
    
    def collate_fn(self, batch):
      batch_x,batch_y = [],[]
      for x,y in batch:
        # print("x shape ",x.shape)
        opening_price = x[:,0].unsqueeze(1)
        open_prev = torch.zeros_like(opening_price)
        for i in range(1,params['context_length']):
          open_prev = torch.zeros_like(opening_price)
          open_prev[i:,:] = opening_price[:-i,:]
          x = torch.cat((x[:,:i],open_prev,x[:,i:]),dim=1)
        batch_x.append(x)
        batch_y.append(y)
      return torch.stack(batch_x),torch.stack(batch_y)

class TestDataset(Dataset):
    def __init__(self):
        self.data = stock_test_inputs_trimmed
        #Delete Volume
        self.data = np.delete(self.data, 1, axis=2)
        self.label = stock_test_labels_trimmed
        self.test_len = self.data.shape[0]
        
    def __len__(self):
        return self.test_len

    def __getitem__(self, index):
      x1 = torch.from_numpy(self.data[index].astype(np.float32))
      x2 = torch.from_numpy(self.label[index].astype(np.float32))
      return x1, x2

    def collate_fn(self, batch):
      batch_x,batch_y = [],[]
      for x,y in batch:
        # print("x shape ",x.shape)
        opening_price = x[:,0].unsqueeze(1)
        open_prev = torch.zeros_like(opening_price)
        for i in range(1,params['context_length']):
          open_prev = torch.zeros_like(opening_price)
          open_prev[i:,:] = opening_price[:-i,:]
          x = torch.cat((x[:,:i],open_prev,x[:,i:]),dim=1)
        batch_x.append(x)
        batch_y.append(y)
      return torch.stack(batch_x),torch.stack(batch_y)

In [ ]:
train_data = TrainDataset()
test_data  = TestDataset()

train_loader = torch.utils.data.DataLoader(train_data, batch_size=params['batch_size'], shuffle=True, collate_fn = train_data.collate_fn)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=params['batch_size'], shuffle=False, collate_fn = test_data.collate_fn)

In [ ]:
# Sanity Check - Plot Windows

import plotly.express as px

def plot_output_windowed(company_data, window_id):
  # 1st index is output (prediction)
  x=np.linspace(1, len(company_data[window_id][1]), num=len(company_data[window_id][1]))
  fig = px.line(x=x, y=company_data[window_id][1])
  fig.show()
  print("op", company_data[window_id][1])

def plot_input_covariate_windowed(company_data, window_id, covariate_index):
  # 0th index is input (the covariates)
  x=np.linspace(1, company_data[window_id][0].shape[0], num=len(company_data[window_id][0][:,covariate_index]))
  fig = px.line(x=x, y=company_data[window_id][0][:,covariate_index])
  fig.show()
  print("ip", company_data[window_id][0][:,covariate_index])

In [ ]:
plot_output_windowed(test_data, 0)
plot_input_covariate_windowed(test_data, 0, 0)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
class Network(nn.Module):
    def __init__(self, params):
        '''
        We define a recurrent network that predicts the 
        future values of a time-dependent variable based on
        past inputs and covariates.
        '''
        super(Network, self).__init__()
        self.params = params
        self.embedding = nn.Embedding(params['num_classes'], params['embedding_dim'])

        self.lstm = nn.LSTM(input_size=params['cov_dim']+params['embedding_dim']+params['context_length']-1,
                            hidden_size=params['lstm_hidden_dim'],
                            num_layers=params['lstm_layers'],
                            bias=True,
                            batch_first=True,
                          )

        self.distribution_mu = nn.Linear(params['lstm_hidden_dim'], 1)
        self.distribution_presigma = nn.Linear(params['lstm_hidden_dim'], 1)
        self.distribution_sigma = nn.Softplus()


    def forward(self, x, h0_c0=None):
        '''
        Predict mu and sigma of the distribution for z_t.
        '''
        cov = x[:, :, :-1]   # remove the company index from the inputs to get the covariates

        company_index = x[:, 0, -1].to(torch.int32)  # retrieve the company index from the covariates
        # print("company_index = ", company_index)
        # print("params['num_classes'] = ", params['num_classes'])
        # print("params['embedding_dim'] = ", params['embedding_dim'])
        onehot_embed = self.embedding(company_index)
        batch_size = cov.shape[0]
        seq_len = cov.shape[1]
        cov_dim = cov.shape[2]
        # print(cov_dim)
        # print(params['cov_dim']+params['context_length']-1)
        assert cov_dim == params['cov_dim']+params['context_length']-1
        assert batch_size <= params['batch_size']

        ohe_embed_all_timestamps = onehot_embed.unsqueeze(1).repeat(1,seq_len,1)

        lstm_input = torch.cat(
            (cov,
             ohe_embed_all_timestamps
            ), dim=2
          )
        
        assert lstm_input.shape[0] <= params['batch_size']
        assert lstm_input.shape[1] == seq_len
        assert lstm_input.shape[2] == params['cov_dim'] + params['embedding_dim']+params['context_length']-1

        out1, hn_cn = self.lstm(input=lstm_input, hx=h0_c0)
              
        out_mu = self.distribution_mu(out1)
        
        out_presigma = self.distribution_presigma(out1)
        out_sigma = self.distribution_sigma(out_presigma)

        return out_mu, out_sigma, hn_cn

In [ ]:
model = Network(params)

In [ ]:
for i, (ip_covariate, op_label) in enumerate(train_loader):
  out_mu, out_sigma, ht_ct = model.forward(x=ip_covariate)

  print('out mu shape', out_mu.shape)
  print('out_sigma shape', out_sigma.shape)
  print('ht shape', ht_ct[0].shape, ht_ct[1].shape)


  break

out mu shape torch.Size([32, 192, 1])
out_sigma shape torch.Size([32, 192, 1])
ht shape torch.Size([4, 32, 256]) torch.Size([4, 32, 256])


In [ ]:
def loss_fn(mu,sigma,labels):
  mu = torch.squeeze(mu)
  sigma = torch.squeeze(sigma)
  total_likelihood = 0
  for i in range(mu.shape[0]):  # loop through batch
    for j in range(mu.shape[1]):# each batch  loop through time steps
      distribution = torch.distributions.normal.Normal(mu[i][j] , sigma[i][j]) # scaling the values by 10 to avoid small sigma values
      total_likelihood += distribution.log_prob(labels[i][j])
  return -1*total_likelihood/(mu.shape[0]*mu.shape[1])

In [ ]:
def loss_fn_updated(mu,sigma,labels):

  batch,seq_len,mudim = mu.shape
  batch,seq_len,sigmadim = sigma.shape

  assert mudim==sigmadim

  mu = mu.view(batch*seq_len,mudim)
  sigma = sigma.view(batch*seq_len,sigmadim)
  labels = labels.view(batch*seq_len,1)

  gauss_prices = torch.distributions.Normal(mu,sigma)
  total_likelihood = -1 * torch.sum(gauss_prices.log_prob(labels))/(batch*seq_len)

  return total_likelihood

In [ ]:
out_price_mu = torch.randn(4, 8, 1)
out_price_sigma = torch.rand(4, 8, 1)
op_label_price = torch.rand(4, 8, 1)

print("orginal loss fn",loss_fn(out_price_mu, out_price_sigma,op_label_price))
print("update loss fn",loss_fn_updated(out_price_mu, out_price_sigma,op_label_price))

orginal loss fn tensor([49.4699])
update loss fn tensor(49.4699)


In [ ]:
def rmse(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sqrt((1/(N*T)) *np.sum((predictions - labels) ** 2))

  denominator = (1/(N*T)) *np.sum(np.abs(labels))
  result =numerator/denominator

  return result

def ND(overall_mu, overall_label):
  predictions = overall_mu.cpu().detach().numpy()
  labels = overall_label.cpu().detach().numpy()

  N, T = predictions.shape
  numerator = np.sum(np.abs(labels - predictions))

  denominator = np.sum(np.abs(labels))
  result =numerator/denominator

  return result

In [ ]:
conditionining_period = params['conditioning_period']
prediction_period     = params['prediction_period']

assert conditionining_period + prediction_period == params['window_size']

In [ ]:
def validate(model):

  model.eval()
  model.cuda()
  total_RMSE = 0

  overall_mu = None
  overall_sigma = None
  overall_label = None

  with torch.no_grad():
    # batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, leave=False, position=0, desc='Test') 

    for i, (ip_covariate, op_label) in enumerate(test_loader):
      ip_covariate = ip_covariate.cuda()
      op_label     = op_label.cuda()

      cond_ip = ip_covariate[:, 0:conditionining_period, :]
      cond_op = op_label[:, 0:conditionining_period]

      pred_ip = ip_covariate[:, conditionining_period: , :]
      pred_op = op_label[:,  conditionining_period: ]

      # Step One - Forward Pass : Conditioning Period
      # print(cond_ip.shape) 
      mu, sigma, ht_ct = model(x=cond_ip, h0_c0=None)

      batch_mu = mu.squeeze()
      batch_sigma = sigma.squeeze()

      # Initialize pred_mu for the first time instance of the "prediction period"
      # from the value of the "predicted mu" from the last CONTEXT LENGTH VALUES of the "conditioning period"
      pred_mu, pred_sigma = mu[:, -params['context_length']:, :], sigma[:, -1, :].unsqueeze(1)
      # print("pred_mu")
      # print(pred_mu)
      # print("pred_mu shape",pred_mu.shape)

      for t in range(0, prediction_period):

        #setting the covariates to the ones recieved from data loader
        pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)
        # print("initial input")
        # print(pred_cov_ip)

        #overwriting the opening price (t-1),(t-2),(t-3)
        for i in range(params['context_length']):
          # print("pred_cov_ip[:, 0, i]",pred_cov_ip[:, 0, i].shape)
          # print("pred_mu[:,:, 0]",pred_mu[:, :, 0].shape)
          pred_cov_ip[:, 0, i] = pred_mu[:, -(i+1), 0].squeeze()
        
        # print("overwritten input")
        # print(pred_cov_ip)
        
        mu, pred_sigma, ht_ct = model(x=pred_cov_ip, h0_c0=ht_ct)
        
        pred_mu = torch.roll(pred_mu, shifts=-1, dims=1)
        pred_mu[:,-1,:] = mu.squeeze(1)
        
        batch_mu = torch.cat((batch_mu,  mu.squeeze(1)), dim=1)
        batch_sigma = torch.cat((batch_sigma, pred_sigma.squeeze(2)), dim=1)

        


      if overall_mu is None and overall_sigma is None:
        overall_mu = batch_mu
        overall_sigma = batch_sigma
        overall_label = op_label
      else:
        overall_mu = torch.cat((overall_mu,  batch_mu), dim=0)
        overall_sigma = torch.cat((overall_sigma, batch_sigma), dim=0)
        overall_label = torch.cat((overall_label, op_label), dim=0)
        
    rmse_val = rmse(overall_mu[:, params['conditioning_period']:], overall_label[:, params['conditioning_period']:])
    nd_val = ND(overall_mu[:, params['conditioning_period']:], overall_label[:, params['conditioning_period']:])

  return rmse_val,nd_val

In [ ]:
def plot_output_data(data, window_id):
  # 1st index is output (prediction)
  x=np.linspace(1, len(data[window_id]), num=len(data[window_id]))
  fig = px.line(x=x, y=data[window_id])
  fig.show()
  # print("op", data[window_id])

In [ ]:
validate(model)

In [ ]:
for i, (ip_covariate, op_label) in enumerate(train_loader):
  print(ip_covariate.shape)
  break

torch.Size([96, 192, 6])


In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

     |████████████████████████████████| 1.8 MB 4.1 MB/s 
     |████████████████████████████████| 181 kB 85.9 MB/s 
     |████████████████████████████████| 144 kB 59.7 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# wandb.init(project="final-results", entity="idl-deepar-spring-22", name="context_without_volume")

In [ ]:
model_version='deepar_model_no_volume_v1.pt'
epochs = params['epochs']

best_dev_rmse = 10000

model = Network(params)
optimizer = torch.optim.Adam(model.parameters(), lr=params['learning_rate'])
#scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5, threshold=0.01, threshold_mode='rel', cooldown=2, min_lr=0, eps=1e-08, verbose=False)
scaler = torch.cuda.amp.GradScaler()
print(model)

for epoch in range(0, epochs):
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    model.train()
    model.cuda()

    total_loss = 0


    for i, (ip_covariate, op_label) in enumerate(train_loader):
        optimizer.zero_grad()

        ip_covariate = ip_covariate.cuda()
        # print(ip_covariate.shape)
        op_label     = op_label.cuda()

        mu, sigma, ht_ct = model(x=ip_covariate, h0_c0=None)

        # print('mu shape', mu.shape, 'sigma shape', sigma.shape, 'op label', op_label.shape)
        loss = loss_fn_updated(mu, sigma, op_label)

        # print("loss", loss)

        total_loss += float(loss)
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update()
        #loss.backward()
        #optimizer.step()

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        batch_bar.update() 

    batch_bar.close() # You need this to close the tqdm bar
    val_rmse, val_nd= validate(model=model)
    scheduler.step(val_rmse)
    running_loss = total_loss/len(train_loader)
    # wandb.log({'lr': optimizer.param_groups[0]['lr'], 'Train_loss': running_loss, 'rmse':val_rmse, 'ND':val_nd })

    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss/len(train_loader),
            },  f'/content/drive/MyDrive/Shreya_Project/{1}')
  

    if val_rmse < best_dev_rmse:
      best_dev_rmse = val_rmse
    print("Epoch {}/{}: Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs,
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])))
    print("rmse is ", val_rmse)

Network(
  (embedding): Embedding(249, 128)
  (lstm): LSTM(140, 256, num_layers=4, batch_first=True)
  (distribution_mu): Linear(in_features=256, out_features=1, bias=True)
  (distribution_presigma): Linear(in_features=256, out_features=1, bias=True)
  (distribution_sigma): Softplus(beta=1, threshold=20)
)


Epoch 1/100: Train Loss -1.6330, Learning Rate 0.0020
rmse is  0.48003882813409704


Epoch 2/100: Train Loss -2.3112, Learning Rate 0.0020
rmse is  0.3787038499750919


Epoch 3/100: Train Loss -2.8045, Learning Rate 0.0020
rmse is  0.33787413829130875


Epoch 4/100: Train Loss -3.0201, Learning Rate 0.0020
rmse is  0.37202484658950036


...

Epoch 16/100: Train Loss -0.4375, Learning Rate 0.0014
rmse is  0.5667925789731042


Epoch 17/100: Train Loss -0.4376, Learning Rate 0.0010
rmse is  0.5544613350433902


Epoch 18/100: Train Loss -0.4386, Learning Rate 0.0010
rmse is  0.5513347850097577


Epoch 19/100: Train Loss -0.4378, Learning Rate 0.0010
rmse is  0.5465643313120782


Train:  93%|█████████▎| 14480/15625 [8:29:13<3:02:15,  9.55s/it, loss=-0.4379, lr=0.0010]

In [ ]:
checkpoint = torch.load(f'/content/drive/MyDrive/DeepARExperiments/deepar_model_no_volume_v1.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

RuntimeError: ignored

In [ ]:
import torch.distributions
def get_pred_sigma_mu_labels(model):

  model.eval()
  model.cuda()


  total_RMSE = 0

  overall_mu = None
  overall_sigma = None
  overall_label = None

  with torch.no_grad():
    # batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, leave=False, position=0, desc='Test') 

    for i, (ip_covariate, op_label) in enumerate(test_loader):
      ip_covariate = ip_covariate.cuda()
      op_label     = op_label.cuda()

      cond_ip = ip_covariate[:, 0:conditionining_period, :]
      cond_op = op_label[:, 0:conditionining_period]

      pred_ip = ip_covariate[:, conditionining_period: , :]
      pred_op = op_label[:,  conditionining_period: ]

      # Step One - Forward Pass : Conditioning Period    
      mu, sigma, ht_ct = model(x=cond_ip, h0_c0=None)


      batch_mu = mu.squeeze()
      batch_sigma = sigma.squeeze()
      

      # Initialize pred_mu for the first time instance of the "prediction period"
      # from the value of the "predicted mu" from the last instance of the "conditioning period"
      pred_mu, pred_sigma = mu[:, -params['context_length']:, :], sigma[:, -1, :].unsqueeze(1) #-> pred_mu [BxTx1]
      
      # m = torch.distributions.Normal(pred_mu, pred_sigma)
      # sample = m.sample()
      # sample_pred = sample.squeeze()
      # batch_sampled = mu.squeeze()
      # batch_sampled[:,-1] = sample.squeeze()

      # print("batch mu initial shape", batch_mu.shape)
      # print("batch sampled initial shape", batch_sampled.shape)

      for t in range(0, prediction_period):
        pred_cov_ip = pred_ip[:, t, :].unsqueeze(1)
        # pred_cov_ip[:, 0, 0] = pred_mu[:, 0, 0]
        for i in range(params['context_length']):
          # print("pred_cov_ip[:, 0, i]",pred_cov_ip[:, 0, i].shape)
          # print("pred_mu[:,:, 0]",pred_mu[:, :, 0].shape)
          pred_cov_ip[:, 0, i] = pred_mu[:, -(i+1), 0].squeeze()

        mu, pred_sigma, ht_ct = model(x=pred_cov_ip, h0_c0=ht_ct)

        pred_mu = torch.roll(pred_mu, shifts=-1, dims=1)
        pred_mu[:,-1,:] = mu.squeeze(1)
        
        batch_mu = torch.cat((batch_mu,  mu.squeeze(2)), dim=1)

        # m = torch.distributions.Normal(pred_mu, pred_sigma)
        # sample = m.sample()
        # sample_pred = sample.squeeze()
        # batch_sampled = torch.cat((batch_sampled,  sample.squeeze(2)), dim=1)

        # print("batch mu updated shape", batch_mu.shape)
        # print("batch sampled updated shape", batch_sampled.shape)



        batch_sigma = torch.cat((batch_sigma, pred_sigma.squeeze(2)), dim=1)


      if overall_mu is None and overall_sigma is None:
        # overall_samples = batch_sampled
        overall_mu = batch_mu
        overall_sigma = batch_sigma
        overall_label = op_label
      else:
        overall_mu = torch.cat((overall_mu,  batch_mu), dim=0)
        # overall_samples = torch.cat((overall_samples,  batch_sampled), dim=0)
        overall_sigma = torch.cat((overall_sigma, batch_sigma), dim=0)
        overall_label = torch.cat((overall_label, op_label), dim=0)
  

    # print("Shape of batch_mu", batch_mu.shape)
    # print("Shape of batch_sampled", batch_sampled.shape)

  return  overall_mu, overall_sigma, overall_label

In [ ]:
import pandas as pd
import plotly.graph_objects as go
def plot_test_output_data(actual,predicted, sigma, window_id):
  print("Predicted = ", predicted)
  x=np.linspace(1, len(actual[window_id]), num=len(actual[window_id]))
  df = pd.DataFrame()
  df['actual'] = actual[window_id]
  df['predicted'] = predicted[window_id]
  #df['sampled']=sampled[window_id]
  df['predicted_upper'] = predicted[window_id] + 1*sigma[window_id]
  df['predcited_lower'] = predicted[window_id] - 1*sigma[window_id]
  # print("df predicted = ", df['predicted'])

  fig = go.Figure()
  fig.add_vline(x=168)
  fig.add_traces(go.Scatter(x=x, y = df['actual'], mode = 'lines', name = 'Actual'))
  fig.add_traces(go.Scatter(x=x, y = df['predicted'], mode = 'lines', name = 'Predicted'))
  #fig.add_traces(go.Scatter(x=x, y = df['sampled'], mode = 'lines', name = 'Sampled'))

  fig.add_traces(go.Scatter(
        name='Upper Bound',
        x=x.tolist(),
        y=df['predicted_upper'],
        mode='lines',
        marker=dict(color="#444"),
        line=dict(width=0),
        showlegend=False
    ))
  fig.add_traces(
    go.Scatter(
        name='Lower Bound',
        x=x.tolist(),
        y= df['predcited_lower'],
        marker=dict(color="#444"),
        line=dict(width=0),
        mode='lines',
        fillcolor='rgba(68, 68, 68, 0.3)',
        fill='tonexty',
        showlegend=False
    ))
  

  fig.update_xaxes(title_text='Time Samples')
  fig.update_yaxes(title_text='Normalized Stock Opening Price')
  fig.show()

In [ ]:
mu_no_volume, sigma_no_volume, label_no_volume =  get_pred_sigma_mu_labels(model)
plot_test_output_data(label_no_volume.cpu().numpy(),mu_no_volume.cpu().numpy(), sigma_no_volume.cpu().numpy(), 11000 
                      
                      )
# plot_test_output_data(label_no_volume.cpu().numpy(),samples_no_volume.cpu().numpy(), sigma_no_volume.cpu().numpy(), 100)

Predicted =  [[ 0.08060318  0.0964447   0.08848543 ... -1.2808992  -1.3600297
  -1.4419509 ]
 [ 0.08421507  0.07817522  0.07991835 ... -1.7238387  -1.875146
  -1.8674041 ]
 [ 0.05651615  0.07341048  0.0761237  ... -1.5437918  -1.5742885
  -1.5820243 ]
 ...
 [ 0.37381244  0.37854525  0.37531224 ...  0.6001734   0.5761881
   0.60553   ]
 [ 0.37621233  0.36282602  0.40395933 ...  0.5463468   0.5756979
   0.60877925]
 [ 0.41492218  0.4015922   0.46570712 ...  0.53073996  0.5435107
   0.51875055]]


In [ ]:
print(mu_no_volume)

tensor([0.7409, 0.7343, 0.7428,  ..., 1.1823, 1.2476, 1.2537], device='cuda:0')
